In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('T_OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hugging_api_key = os.getenv('HUGGINGFACEHUB_API_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_api_key

In [2]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [3]:
llm = OpenAI(
    model_name='text-ada-001',
    temperature=0,
    openai_api_key=openai_api_key
    )

In [4]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [5]:
conversation = ConversationChain(
    llm=llm,
)

In [6]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [7]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            {"name": self.__class__.__name__},
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except (KeyboardInterrupt, Exception) as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_

In [8]:
# add buffer memory for conversation chain
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [9]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!', 'history': '', 'response': '\n\nGood morning!'}

In [10]:
count_tokens(
    conversation_buf,
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 116 tokens


"\n\nThat's an interesting idea! I'm not sure how it would work, but it's an interesting idea."

# Memory de ne varsa her seferinde token olarak openai dan cevap almaya kullaniyor

In [11]:
count_tokens(
    conversation_buf,
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 248 tokens


' \n\nI can think of a few possibilities where large language models could be integrated with external knowledge. One possibility would be to create a system that could understand complex texts and understand them in a more detail-able way. This would be great for those who need to communicate complex topics. Another possibility is to use the data collected from understanding texts to create new technologies that can help with the field of security. It could be that after understanding the text, the system could produce differently shaped objects or symbols that could be used in security videos or applications.'

In [12]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI: 

Good morning!
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI: 

That's an interesting idea! I'm not sure how it would work, but it's an interesting idea.
Human: I just want to analyze the different possibilities. What can you think of?
AI:  

I can think of a few possibilities where large language models could be integrated with external knowledge. One possibility would be to create a system that could understand complex texts and understand them in a more detail-able way. This would be great for those who need to communicate complex topics. Another possibility is to use the data collected from understanding texts to create new technologies that can help with the field of security. It could be that after understanding the text, the system could produce differently shaped objects or symbols that could be used in security videos or applications.


In [13]:
conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

In [14]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [15]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum,
    "Good morning AI!"
)

Spent a total of 255 tokens


'\n\nGood morning!'

In [16]:
count_tokens(
    conversation_sum,
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 396 tokens


"\n\nThat's an interesting idea! I'm not sure if this is the right time or place, but I can give you some ideas on how we could go about this."

In [17]:
# initialize tokenizer
tokenizer = tiktoken.encoding_for_model('text-ada-001')

# show number of tokens for the memory used by each memory type
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}'
)

Buffer memory conversation length: 194
Summary memory conversation length: 32
